In [1]:
%%capture
!pip install pyctcdecode
!python -m pip install pypi-kenlm
!pip install jiwer

![](https://developer-blogs.nvidia.com/wp-content/uploads/2019/12/automatic-speech-recognition_updated.png)

![](https://www.researchgate.net/profile/Diana-Militaru/publication/299594444/figure/fig1/AS:346834426974208@1459703179403/The-block-diagram-of-an-automatic-speech-recognition-and-understanding-system.png)

in this notebook we will try to demonstrate how to calculate CER,WER metric on validation dataset using xls-r wav2vec2 model,we will be using public best available pretrained model from huggingface to demonstrate the metric calculation process. for understanding how to train wav2vec2 on this dataset please check our past work [wav2vec2 starter](https://www.kaggle.com/code/nazmuddhohaansary/wave2vec2-starter-for-dl-sprint-commonvoice)

# Imports

In [2]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import re
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps

tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



# Configs

In [3]:
#according to our experiment this is the best model -> arijitx/wav2vec2-xls-r-300m-bengali
class CFG:
    model_name = 'arijitx/wav2vec2-xls-r-300m-bengali' #arijitx/wav2vec2-large-xlsr-bengali,arijitx/wav2vec2-xls-r-300m-bengali, Tahsin-Mayeesha/wav2vec2-bn-300m
    valid_df_path = '../input/dlsprint/validation.csv'
    sample_sub_df_path = '../input/dlsprint/sample_submission.csv'
    valid = "../input/dlsprint/validation_files/"
    test = "../input/dlsprint/test_files/"
    valid_wav = '../input/validation-fileswav-format/validation_files_wav/'
    test_wav = '../input/test-wav-files-dl-sprint/test_files_wav/'
    batch_size = 48#not using this param now
    single_SPEECH_FILE = "../input/dlsprint/validation_files/common_voice_bn_30620258.mp3"
    



# single sample inference demo

In [4]:
asr = pipeline("automatic-speech-recognition", model=CFG.model_name, device=0)
feature_extractor = AutoFeatureExtractor.from_pretrained(
        CFG.model_name, cache_dir=None, use_auth_token=False
    )
speech, sr = librosa.load(CFG.single_SPEECH_FILE, sr=feature_extractor.sampling_rate)
prediction = asr(
            speech, chunk_length_s=112, stride_length_s=None
        )

pred = prediction["text"]
pred


Downloading:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.48G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/57.2M [00:00<?, ?B/s]

'তার পিতার নাম কালীপ্রসন্ন ভট্টাচার্য'

# check the original audio

In [5]:
IPython.display.Audio(CFG.single_SPEECH_FILE)

# Fix paths

In [6]:
df = pd.read_csv('../input/dlsprint/validation.csv')
directory ="../input/dlsprint/validation_files/"
df["path"]=df["path"].progress_apply(lambda x:os.path.join(directory,str(x)))
df.head(3)

  0%|          | 0/7747 [00:00<?, ?it/s]

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale
0,c0494c8220a53efec93f188e32be94d3c1832c48117423...,../input/dlsprint/validation_files/common_voic...,"কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।",3.0,0.0,NaN,NaN,NaN,bn
1,c0494c8220a53efec93f188e32be94d3c1832c48117423...,../input/dlsprint/validation_files/common_voic...,তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।,6.0,1.0,NaN,NaN,NaN,bn
2,c06b36547c86713d53bb2bf696a34b696de586c5ab1aa9...,../input/dlsprint/validation_files/common_voic...,ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষ...,3.0,1.0,NaN,NaN,NaN,bn


# Custom dataset class

librosa with mp3 is super slow,so we will be using wav files for faster inference

In [7]:
class bn_asr_Dataset(Dataset):
    '''
    args:
        df      : path of the dataframe
        dir     : directory of sound files
    '''
    def __init__(self,df,dir):
        self.df = pd.read_csv(df)
        self.dir = dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
   
        #speech, _ = librosa.load(self.dir+self.df.path[i], sr=feature_extractor.sampling_rate) 
        path = self.dir+self.df.path[i]
        path = os.path.splitext(path)[0]+'.wav'
        # Read file
        sampling_rate, data = wavfile.read(path)
        # Resample data
        number_of_samples = round(len(data) * float(feature_extractor.sampling_rate) / sampling_rate)
        speech = sps.resample(data, number_of_samples)
        return speech
  


# making prediction on whole validation set

In [8]:
%%time
#single image inference
''' 
#super slow inference...

predictions = []
references = []
for i in range(len(df.path)):
    speech, sr = librosa.load(df.path[i], sr=feature_extractor.sampling_rate)
    prediction = asr(speech, chunk_length_s=112, stride_length_s=None)
    pred = prediction["text"]
    predictions.append(pred)
    references.append(df.sentence[i])
    
print(len(predictions),len(references))
'''

df = pd.read_csv(CFG.valid_df_path)
valid_dataset = bn_asr_Dataset(CFG.valid_df_path,CFG.valid_wav)#CFG.valid
predictions = []
references = []
# for i,pred_sentence in enumerate(tqdm(asr(valid_dataset, chunk_length_s=112, stride_length_s=None,batch_size=CFG.batch_size), total=len(valid_dataset))):
#     references.append(df.sentence[i])
#     predictions.append(pred_sentence['text'])
    
for i in range(len(valid_dataset)):
    pred = asr(valid_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    references.append(df.sentence[i])
    predictions.append(pred['text'])
  

CPU times: user 21min 42s, sys: 8.02 s, total: 21min 50s
Wall time: 24min 33s


In [9]:
torch.cuda.empty_cache() 
gc.collect()
!nvidia-smi

Fri Jul 15 09:01:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    42W / 250W |   2167MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# WER (word error rate) calculation process



![](https://miro.medium.com/max/700/1*MUGLdWm3zMYK7dLmyo3pqA.png)

**WER = 100 (insertions(INS) + substitutions(SUB) + deletions(DEL))**

![](http://www.italk2learn.eu/wp-content/uploads/2015/02/speech-bubble-image.png)

# CER (character error rate) calculation process



character error rate (cer) is a common metric of the performance of an automatic speech recognition system. This value indicates the percentage of characters that were incorrectly predicted. The lower the value, the better the performance of the ASR system with a CER of 0 being a perfect score.

CER calculation is based on the concept of [Levenshtein distance](https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510#9bd1), where we count the minimum number of character-level operations required to transform the ground truth text (aka reference text) into the OCR output.

Character Error Rate (CER) formula :

![](https://miro.medium.com/max/700/1*KsWFDKnLI7mudmhbzGjc4w.png)

where:

* S = Number of Substitutions
* D = Number of Deletions
* I = Number of Insertions
* N = Number of characters in reference text (aka ground truth)

Let’s look at an example:

**Ground Truth Reference Text**: 809475127

**ASR Transcribed Output Text**: 80g475Z7

Several errors require edits to transform ASR output into the ground truth:

1. g instead of 9 (at reference text character 3)
2. Missing 1 (at reference text character 7)
3. Z instead of 2 (at reference text character 8)

With that, here are the values to input into the equation:

* Number of Substitutions (S) = 2
* Number of Deletions (D) = 1
* Number of Insertions (I) = 0
* Number of characters in reference text (N) = 9

Based on the above, we get (2 + 1 + 0) / 9 = 0.3333. When converted to a percentage value, the CER becomes 33.33%. This implies that every 3rd character in the sequence was incorrectly transcribed.

We repeat this calculation for all the pairs of transcribed output and corresponding ground truth, and take the mean of these values to obtain an overall CER percentage.

**Reference :** [Evaluate OCR Output Quality with Character Error Rate (CER) and Word Error Rate (WER)](https://towardsdatascience.com/evaluating-ocr-output-quality-with-character-error-rate-cer-and-word-error-rate-wer-853175297510#5aec)

# calculating metric on whole validation set

In [10]:

df = pd.DataFrame(columns=['predictions', 'references'])
df.predictions = predictions
df.references = references
df.to_csv('./results.csv',index = False) #use it for error analysis and other stuffs
df.head(10)

,predictions,references
0,কৃষি সেবা রেমিটেন্স ব্যবসা ও অন্যান্য,"কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।"
1,তিনি ছিলেন চাকমা বাসার প্রথ মাধ্যমের গীতিকা।,তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।
2,ইংরেজি সাথে সাথে তাদের হিন্দি ও সংস্কৃত শিক্ষা...,ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষ...
3,শিক্ষার ধারণ তার প্রথম আন্তর্জাতিক উইক েটশিকার...,শিখর ধাওয়ান তার প্রথম আন্তর্জাতিক উইকেট শিকার...
4,চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে কিন...,"চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন..."
5,এখানে তিনি এয়াকিনসের অধীনে অধ্যয়ন করেছেন,এখানে তিনি এয়াকিনসের অধীনে অধ্যয়ন করেছিলেন।
6,শীতকালীন গেমস এ কোন কোন পদক জিততে পারেনি।,শীতকালীন গেমসে এখনো কোন পদক জিততে পারেনি।
7,অবশেষে তিনি কাবুলে ফিরে আসেন,"অবশেষে, তিনি কাবুলে ফিরে আসেন।"
8,তিনি অ্যাথেলসেএসে চক্রটির শীর্ষ হন,তিনি এথেন্সে এসে সক্রেটিসের শিষ্য হন।
9,তিন বছর বয়সে তার বাবা মারা যান,তিন বছর বয়সে তাঁর বাবা মারা যান।


# Without Post Processing

In [11]:
cer = load_metric("cer")
wer = load_metric("wer")

cer_score = cer.compute(predictions=predictions, references=references)
print("validation cer_score -> ",cer_score)
wer_score = wer.compute(predictions=predictions, references=references)
print("validation wer_score -> ",wer_score)

validation cer_score ->  0.09787704766628824
validation wer_score ->  0.30921300101701055


# With  post processing

during error analysis using the results.csv file we've seen that the model is frequently missing to predict punctuations, almost all the sentences in ground truth ends with '।' but while predicting using the public best trained model we can see that the model is missing to predict '।' most of the times, so in the simple post processing code below we will check if the predicted sentence ends with '।' or not,if no then we forcefully add '।' at the end of the predicted sentence.

In [12]:
for i in range(len(predictions)):
    if(predictions[i][-1] == '।'):
        continue
    else:
        predictions[i] = predictions[i]+'।'

In [13]:
cer_score = cer.compute(predictions=predictions, references=references)
print("Final validation cer_score -> ",cer_score)
wer_score = wer.compute(predictions=predictions, references=references)
print("Final validation wer_score -> ",wer_score)

Final validation cer_score ->  0.09301847750965592
Final validation wer_score ->  0.28501372267654884


**WOW great!!!
with above post processing word error rate improved from 0.30921300101701055 to 0.28501372267654884 and that's 0.024199278340461705 improvement,not bad no?**


# Submission with post processing

In [14]:
df = pd.read_csv('../input/dlsprint/sample_submission.csv')
len(df.path)

7747

In [15]:
%%time

test_dataset = bn_asr_Dataset(CFG.sample_sub_df_path,CFG.test_wav)

# for i,prediction in enumerate(tqdm(asr(test_dataset, chunk_length_s=112, stride_length_s=None,batch_size=CFG.batch_size), total=len(test_dataset))):
#     df.sentence[i] = prediction["text"]
    
for i in range(len(test_dataset)):
    pred = asr(test_dataset.__getitem__(i), chunk_length_s=112, stride_length_s=None)
    
    #applying simple post processing with error handler
    try:
        if(pred["text"][-1] == '।'):
            df.sentence[i] = pred["text"]
        else:
            df.sentence[i] = pred["text"]+'।'
    except:
        print("predicted text at idx ",i," is -> ",pred["text"])
        df.sentence[i] = pred["text"]+'।'


predicted text at idx  3936  is ->  
CPU times: user 23min 13s, sys: 8.55 s, total: 23min 21s
Wall time: 26min 3s


In [16]:
df.head(3)

,path,sentence
0,common_voice_bn_31675220.mp3,এছাড়াও নিউজল্যান্ড এই ক্রিকেট দলের হয়ে খেলছে...
1,common_voice_bn_31513116.mp3,এই ফল পাখি রাখায় কিন্তু নিজে পড়ে থাকা ফল খেল...
2,common_voice_bn_31558126.mp3,জন পরিকল্পিত।


In [17]:
df.to_csv('./submission.csv',index = False)
df.sentence[1]

'এই ফল পাখি রাখায় কিন্তু নিজে পড়ে থাকা ফল খেলে কুকুর অসুস্থ হয়ে পড়ে।'

In [18]:
IPython.display.Audio('../input/dlsprint/test_files/common_voice_bn_31675220.mp3')

In [19]:
df.sentence[0]

'এছাড়াও নিউজল্যান্ড এই ক্রিকেট দলের হয়ে খেলছেন তিনি।'

In [20]:
df.sentence[80]

'প্রথম শ্রেণির ক্রিকেট প্রতিযোগিতা শেলিল্ডের উদ্বোধনী আসরে অংশ নেয়।'

# optional (post ASR correction attempt)

in this section we will try to implement the recent best research on POST OCR (optical character recognition) CORRECTION titled[ Post-OCR Document Correction with large Ensembles of Character
Sequence-to-Sequence Models](https://arxiv.org/pdf/2109.06264.pdf) this research work was done in ocr domain and not in ASR domain so i was thinking what will happen if we try this approach in ASR domain? **well if you never try you'll never know**.
The core of this system is a standard sequence-to-sequence model that can correct sequences of characters. In the below implementation, we used a Transformer as the sequence model, which takes as input a segment of characters from the document to correct, and the output is the corrected segment. To train this sequence model, it is necessary to align the raw documents with their corresponding correct transcriptions, which is not always straightforward.Since the output is not necessarily of the same length as the input (because of possible insertions or deletions of characters), a decoding method like Greedy Search or Beam Search
is needed to produce the most likely corrected sequence according to the model.
for the below experiment we will be using results.csv where references column contains actual clean annotation and predictions column contains output of STT model including errors


In [21]:
!git clone https://github.com/jarobyte91/post_ocr_correction.git
os.chdir('./post_ocr_correction')
!pwd
!ls

Cloning into 'post_ocr_correction'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 225 (delta 11), reused 6 (delta 2), pack-reused 203
Receiving objects: 100% (225/225), 287.00 KiB | 901.00 KiB/s, done.
Resolving deltas: 100% (107/107), done.
/kaggle/working/post_ocr_correction
LICENSE    download_data.py  lib	pyproject.toml	  setup.cfg  train
README.md  evaluate	     notebooks	requirements.txt  tests


In [22]:
!pip install .

Processing /kaggle/working/post_ocr_correction
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Installing backend dependencies ... - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.4 MB/s eta 0:00:00
  Created wheel for post-ocr-correction: filename=post_ocr_correction-0.0.1-py3-none-any.whl size=6368 sha256=c930612188cd501a6221081678f0ada3cc1d2367ea158f52fb78f421399c16fa
  Stored in directory: /tmp/pip-ephem-wheel-cache-s6_53y6f/wheels/72/31/2e/2f7f310c7a5d2c12d18796bf8b5e2a7a79569dcc1f6f07f977
Successfully built post-ocr-correction
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [23]:
os.chdir('..')
!ls

__notebook__.ipynb  post_ocr_correction  results.csv  submission.csv


In [24]:
results = pd.read_csv('./results.csv')
print(len(results))
preds = results.predictions.tolist()
refs = results.references.tolist()
results.head()

7747


,predictions,references
0,কৃষি সেবা রেমিটেন্স ব্যবসা ও অন্যান্য,"কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।"
1,তিনি ছিলেন চাকমা বাসার প্রথ মাধ্যমের গীতিকা।,তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।
2,ইংরেজি সাথে সাথে তাদের হিন্দি ও সংস্কৃত শিক্ষা...,ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষ...
3,শিক্ষার ধারণ তার প্রথম আন্তর্জাতিক উইক েটশিকার...,শিখর ধাওয়ান তার প্রথম আন্তর্জাতিক উইকেট শিকার...
4,চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে কিন...,"চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন..."


Be careful,
if you predict on train set using the best publicly available ASR bangla model from huggingface you will see the model making NaN prediction for many audio samples in train set,to get the index of those NaN output files i used the code below 

In [25]:
#no nan output in results.csv but they exist in train.csv (give it a try)
idx = [i for i, x in zip(range(len(preds)), preds) if not isinstance(x,str)]
for ele in sorted(idx, reverse = True):
    del preds[ele]
    del refs[ele]
len(preds),len(refs)


(7747, 7747)

In [26]:
torch.cuda.empty_cache()
gc.collect()
!nvidia-smi

Fri Jul 15 09:28:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    44W / 250W |   2167MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [27]:
from pytorch_beam_search import seq2seq
from post_ocr_correction import correction

# train data and model
source = [preds]
target = [refs]
source_index = seq2seq.Index(source)
target_index = seq2seq.Index(target)
X = source_index.text2tensor(source)
Y = target_index.text2tensor(target)
print(source_index)
print(".....")
print(target_index)
print(".....")
print(X)
print(".....")
print(Y)

<Seq2Seq Index with 7,750 items>
.....
<Seq2Seq Index with 7,751 items>
.....
tensor([[   1, 2683, 4223,  ..., 1101, 2674,    2]])
.....
tensor([[   1, 2716, 4287,  ..., 1009, 2705,    2]])



![](https://i.ibb.co/741XzvD/post-stt-corrector.png)

In [28]:
%%time

epochs = 75
PATH = './post_ASR_corrector.pt'

def post_asr_corrector():
    model = seq2seq.Transformer(source_index, target_index,max_sequence_length = len(results)+4,
                    embedding_dimension = 512,
                    feedforward_dimension = 1024,
                    attention_heads = 2,
                    encoder_layers = 2,
                    decoder_layers = 2
                           )
    return model
model = post_asr_corrector()


model.train()
model.fit(X, Y, epochs = epochs, progress_bar = 1)
model.eval()
torch.save(model.state_dict(), PATH)

Model: Seq2Seq Transformer
Source index: <Seq2Seq Index with 7,750 items>
Target index: <Seq2Seq Index with 7,751 items>
Max sequence length: 7751
Embedding dimension: 512
Feedforward dimension: 1024
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 26,397,767



  0%|          | 0/75 [00:00<?, ?it/s]

Training started
X_train.shape: torch.Size([1, 7749])
Y_train.shape: torch.Size([1, 7749])
Epochs: 75
Learning rate: 0.0001
Weight decay: 0
Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   9.1148 |     99.987 |     1.2
    2 |   9.0522 |     99.948 |     2.3
    3 |   8.9953 |     99.935 |     3.4
    4 |   8.9376 |     99.961 |     4.5
    5 |   8.8774 |     99.961 |     5.6
    6 |   8.8152 |     99.948 |     6.7
    7 |   8.7510 |     99.935 |     7.8
    8 |   8.6855 |     99.845 |     8.9
    9 |   8.6195 |     99.768 |    10.0
   10 |   8.5533 |     99.664 |    11.1
   11 |   8.4871 |     99.561 |    12.2
   12 |   8.4202 |     99.380 |    13.3
   13 |   8.3530 |     99.161 |    14.4
   14 |   8.2857 |     98.761 |    15.5
   15 |   8.2182 |     98.309 |    16.6
   16 |   8.1505 |     97.625 |    17.7
   17 |   8.0826 |     96.825 |    18.8
   18 |   8.0143 |     95.909 |    19.9
   19 |   7.9458 |     94.95

# load and infer

In [29]:
del model
torch.cuda.empty_cache()
gc.collect()


model = post_asr_corrector()

model.load_state_dict(torch.load(PATH))
model.eval()
model

Model: Seq2Seq Transformer
Source index: <Seq2Seq Index with 7,750 items>
Target index: <Seq2Seq Index with 7,751 items>
Max sequence length: 7751
Embedding dimension: 512
Feedforward dimension: 1024
Encoder layers: 2
Decoder layers: 2
Attention heads: 2
Activation: relu
Dropout: 0.0
Trainable parameters: 26,397,767



Transformer(
  (source_embeddings): Embedding(7750, 512)
  (target_embeddings): Embedding(7751, 512)
  (positional_embeddings): Embedding(7751, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (linear2): Linear(in_features=1024, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.0, inplace=False)
          (dropout2): Dropout(p=0.0, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj

In [30]:
# test data
test = "কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।"
reference = "কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্য"
new_source = [list(test)]
X_new = source_index.text2tensor(new_source)

# plain beam search
predictions, log_probabilities = seq2seq.beam_search(
    model, 
    X_new,
    progress_bar = 0
)
just_beam = target_index.tensor2text(predictions[:, 0, :])[0]
just_beam = re.sub(r"<START>|<PAD>|<UNK>|<END>.*", "", just_beam)

In [31]:
log_probabilities

tensor([[-80.6727, -82.7041, -82.8608, -82.9353, -83.0217]])

In [32]:
# post ocr correction
disjoint_beam = correction.disjoint(
    test,
    model,
    source_index,
    target_index,
    50,
    "beam_search",
)

In [33]:
print("\nresults")
print("  test data                      ", test)
print("  plain beam search              ", just_beam)
print("  disjoint windows, beam search  ", disjoint_beam)
print("  disjoint_beam[0:len(test)] ->  ", disjoint_beam[0:len(test)]) 


results
  test data                       কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।
  plain beam search               কৃষি, সেবা, রেমিটেন্স, ব্যবসা ও অন্যান্য।তিনি ছিলেন চাকমা ভাষার প্রথম আধুনিক গীতিকার।ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষা দেওয়া হতো।শিখর ধাওয়ান তার প্রথম আন্তর্জাতিক উইকেট শিকারে পরিণত হয়েছিলেন।চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন্তু এই উন্নতির হারের কিছু হেরফের আছে।এখানে তিনি এয়াকিনসের অধীনে অধ্যয়ন করেছিলেন।শীতকালীন গেমসে এখনো কোন পদক জিততে পারেনি।অবশেষে, তিনি কাবুলে ফিরে আসেন।তিনি এথেন্সে এসে সক্রেটিসের শিষ্য হন।তিন বছর বয়সে তাঁর বাবা মারা যান।এর আগে রাজারহাট কেন্দ্রটি বিদ্যমান ছিল না।অবসর পরবর্তীকালে আম্পায়ার ও ধারাভাষ্যকারের ভূমিকায় অবতীর্ণ হন।এই বৃদ্ধি আগামী চল্লিশ থেকে পঞ্চাশ বৎসর ধরে ঘটতে থাকবে।নিষিদ্ধ আদেশ তৎক্ষণাৎ শহরের উপর চাপিয়ে দেওয়া হয়েছিল।বর্তমানে তিনি মার্কিন যুক্তরাষ্ট্রের ফ্লোরিডায় বসবাস করছেন।সর্বত্রই কাঁচা কর্দমাক্ত রাস্তা এসব রাস্তা দিয়েই মানুষ অতি কষ্টে চলাচল করত।সত্যি এ বাড়িতে কোন মহিলা খুন হয়েছিল?এটি নির্মাণ কর

# improvement ideas

for better post ASR correction,i would like to recommend going deep in [ROBART](https://arxiv.org/pdf/2202.01157.pdf)
![](https://i.ibb.co/YWVGRVF/post-asr-corrector.png)

![](https://i.ibb.co/9sZbvD7/post-asr.png)
one example implementation of levenshtein transformer can be found [here](https://github.com/nmfisher/levenshtein_transformer/blob/master/Untitled.ipynb)

more about post ASR correction was discussed [here](https://www.kaggle.com/competitions/dlsprint/discussion/335411)

![](https://images.unsplash.com/photo-1499744937866-d7e566a20a61?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=870&q=80)